In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import math
from IPython.display import SVG
import matplotlib.pyplot as plt
import os
import json
import keras
import tensorflow as tf

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15330142053481496252
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3282324684
locality {
  bus_id: 1
}
incarnation: 5161312160380008554
physical_device_desc: "device: 0, name: GeForce GTX 950M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:

params = {}
params["test_dir"] = "test"
params["batch_size"] = 64
params["img_width"] = 221
params["img_height"] = 221
params["workers"] = 8
params["model_file"] = "best_model.h5"

with open("params_evaluate.json", "w") as f:
    json.dump(params, f)

In [5]:
params = {}

with open("params_evaluate.json", "r") as f:
    params = json.load(f)

params["test_size"] = sum([len(files) for r, d, files in os.walk( params["test_dir"] )])

In [7]:
val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
        params["test_dir"],
        target_size = (params["img_width"], params["img_height"]),
        batch_size = params["batch_size"],
        shuffle = True,
        class_mode = 'categorical')

Found 957 images belonging to 102 classes.


In [18]:

sess = tf.Session()
K.set_session(sess)
params["historypath"] = "./history/evaluation_mode"

if not os.path.exists(params['historypath']):
    os.makedirs(params['historypath'])
            
model = keras.models.load_model(params["model_file"])
history = model.evaluate_generator(generator = validation_generator,
                                steps=math.ceil(params["test_size"]  / params["batch_size"]),
                                max_queue_size=10, 
                                workers=params["workers"],
                                use_multiprocessing=False)            
final_history = {}
count = 0
for i in model.metrics_names:
    final_history[i] = history[count]
    count += 1
with open(params["historypath"] + "/history.json", "w") as f:
        json.dump(final_history, f)  
                
del model
sess.close()
tf.reset_default_graph()

K.clear_session()